In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from model_mo2cap2 import mo2cap_ours
from tqdm import tqdm
import os

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = mo2cap_ours(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		image = test_batch['image'].cuda()
		image_zoom = test_batch['image_zoom'].cuda()
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {
			'image':image,
			'image_zoom' : image_zoom,
			}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp806'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp805/0.pth',
 '/workspace/2d_to_3d/apps/exp805/1.pth',
 '/workspace/2d_to_3d/apps/exp805/2.pth',
 '/workspace/2d_to_3d/apps/exp805/3.pth',
 '/workspace/2d_to_3d/apps/exp805/4.pth',
 '/workspace/2d_to_3d/apps/exp805/5.pth',
 '/workspace/2d_to_3d/apps/exp805/6.pth',
 '/workspace/2d_to_3d/apps/exp805/7.pth',
 '/workspace/2d_to_3d/apps/exp805/8.pth',
 '/workspace/2d_to_3d/apps/exp805/9.pth',
 '/workspace/2d_to_3d/apps/exp805/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	_path = f'/workspace/2d_to_3d/apps/eval/mo2cap2_depth_sil'
	_name = f'baseline806_{i}.json'
	if not os.path.exists(_path):
		os.makedirs(_path)

	main(model_path=p,output_path=os.path.join(_path,_name))

08:24:08 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 229.4725799560547, 'UpperStretching': 229.2767333984375, 'Greeting': 182.7737274169922, 'Walking': 241.0973663330078, 'LowerStretching': 241.79649353027344, 'Patting': 220.62734985351562, 'Reacting': 214.05921936035156, 'Gaming': 213.15432739257812, 'Gesticuling': 158.8573760986328, 'Talking': 174.86419677734375}, 'UpperBody': {'All': 196.50753784179688, 'UpperStretching': 203.3564453125, 'Greeting': 185.2676239013672, 'Walking': 214.52598571777344, 'LowerStretching': 176.54144287109375, 'Patting': 180.3197784423828, 'Reacting': 180.87075805664062, 'Gaming': 168.92929077148438, 'Gesticuling': 149.0716094970703, 'Talking': 164.1964569091797}, 'LowerBody': {'All': 262.4375915527344, 'UpperStretching': 255.197021484375, 'Greeting': 180.27981567382812, 'Walking': 267.6687316894531, 'LowerStretching': 307.0515441894531, 'Patting': 260.9349060058594, 'Reacting': 247.2477264404297, 'Gaming': 257.3793640136719, 'Gesticuling': 168.64309692382812, 'Talking': 185.531906127929

  7%|▋         | 400/5352 [01:36<17:28,  4.72it/s]

{'FullBody': {'All': 230.57550048828125, 'UpperStretching': 230.52993774414062, 'Greeting': 180.7251739501953, 'Walking': 242.321044921875, 'LowerStretching': 242.40704345703125, 'Patting': 225.50628662109375, 'Reacting': 213.83517456054688, 'Gaming': 217.59640502929688, 'Gesticuling': 163.51651000976562, 'Talking': 185.2012939453125}, 'UpperBody': {'All': 196.71609497070312, 'UpperStretching': 203.29637145996094, 'Greeting': 181.6884307861328, 'Walking': 217.26219177246094, 'LowerStretching': 176.74884033203125, 'Patting': 178.84274291992188, 'Reacting': 181.62655639648438, 'Gaming': 168.38722229003906, 'Gesticuling': 151.23028564453125, 'Talking': 162.10809326171875}, 'LowerBody': {'All': 264.43487548828125, 'UpperStretching': 257.7635192871094, 'Greeting': 179.7619171142578, 'Walking': 267.3798828125, 'LowerStretching': 308.0652770996094, 'Patting': 272.1697998046875, 'Reacting': 246.04379272460938, 'Gaming': 266.8056335449219, 'Gesticuling': 175.802734375, 'Talking': 208.2945098876

 11%|█         | 600/5352 [02:19<17:45,  4.46it/s]

...

 93%|█████████▎| 5000/5352 [18:17<01:24,  4.16it/s]

{'FullBody': {'All': 131.17025756835938, 'UpperStretching': 138.75238037109375, 'LowerStretching': 128.00689697265625, 'Gaming': 109.79070281982422, 'Patting': 116.99626159667969, 'Talking': 87.49148559570312, 'Reacting': 122.44300842285156, 'Walking': 120.32225036621094, 'Greeting': 90.84337615966797, 'Gesticuling': 100.74850463867188}, 'UpperBody': {'All': 113.77539825439453, 'UpperStretching': 127.48403930664062, 'LowerStretching': 90.03766632080078, 'Gaming': 102.49929809570312, 'Patting': 91.74928283691406, 'Talking': 81.53124237060547, 'Reacting': 121.17433166503906, 'Walking': 90.22700500488281, 'Greeting': 79.29553985595703, 'Gesticuling': 113.45970916748047}, 'LowerBody': {'All': 148.5651397705078, 'UpperStretching': 150.02072143554688, 'LowerStretching': 165.9761505126953, 'Gaming': 117.08209228515625, 'Patting': 142.24325561523438, 'Talking': 93.45172882080078, 'Reacting': 123.7116928100586, 'Walking': 150.41751098632812, 'Greeting': 102.3912124633789, 'Gesticuling': 88.0373

 97%|█████████▋| 5200/5352 [19:01<00:35,  4.26it/s]

{'FullBody': {'All': 131.14633178710938, 'UpperStretching': 138.72097778320312, 'LowerStretching': 127.97781372070312, 'Gaming': 109.52423858642578, 'Patting': 116.57657623291016, 'Talking': 87.49771118164062, 'Reacting': 122.67498016357422, 'Walking': 120.33719635009766, 'Greeting': 90.4530258178711, 'Gesticuling': 100.8255386352539}, 'UpperBody': {'All': 113.80140686035156, 'UpperStretching': 127.52530670166016, 'LowerStretching': 90.0130386352539, 'Gaming': 102.28109741210938, 'Patting': 91.5462417602539, 'Talking': 81.4848403930664, 'Reacting': 121.27198028564453, 'Walking': 90.30973815917969, 'Greeting': 78.93299102783203, 'Gesticuling': 113.59651947021484}, 'LowerBody': {'All': 148.49130249023438, 'UpperStretching': 149.91665649414062, 'LowerStretching': 165.94259643554688, 'Gaming': 116.76736450195312, 'Patting': 141.60693359375, 'Talking': 93.51058959960938, 'Reacting': 124.0779800415039, 'Walking': 150.3646697998047, 'Greeting': 101.97305297851562, 'Gesticuling': 88.0545578002

100%|██████████| 5352/5352 [19:34<00:00,  4.56it/s]
11:59:53 [INFO] Main: Saving evaluation results...
11:59:53 [INFO] Main: Done.
